In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as sk
import scipy 
import igraph
import ts2vg # Time series to visibility graphs
import scipy.spatial.distance as distance
import scipy
import time
import matplotlib.pyplot as plt

# Classes and Functions

In [3]:
def threshold(adj,threshold):
    """
    Parameters:
    -----------
        adj - Dicionário cuja chave é o índice dos pontos comparados e o valores é a resultante da medida de
        distância/similaridade utilizada. 
        
        threshold - limiar a ser aplicado.
    Returns:
    --------
        Nova lista de adjacência com os valores abaixo do limiar definido.
    """
    new_adj = {}
    for i,j in adj:
        if (adj[i,j] <= threshold):
            new_adj[i,j] = adj[i,j]
    return(new_adj)
    
def geographical_distance(adj,pos):
    """
    Parameters:
    -----------
        adj - pares de pontos que correspondem a um ponto (série temporal) disposta no espaço geográfico da 
        bacia do rio Tamanduateí.
        pos - índice (i,j) de cada ponto na matriz que representa o espaço geográfico.
        
    Returns:
    --------
        A distância euclidiana entre os pontos distribuídos na região analisada.
    """
    geo_dist = {}
    for i in adj:
        a = pos[i[0]]
        b = pos[i[1]]
        dist = np.sqrt((a[0]-b[0])**2+(a[1]-b[1])**2)
        geo_dist[i] = dist
    return(geo_dist)

def norm(data):
    """
    Parameters:
    -----------
        Vetor numérico.
    
    Returns:
    --------
        Valores da vetor data entre o intervalo de 0 e 1.
    
    """
    n = []
    max_value = max(data)
    min_value = min(data)
    for i in range(len(data)):
        n.append((data[i]-min_value)/(max_value - min_value))
    return(n)

class reading():
    
    def files():
        """
        Returns:
        --------
            Dados meteorológicos relacionados à região que compreende a bacia do rio Tamanduateí:
            4464 arquivos em formato de matriz com 37 linhas e 36 colunas contendo a previsão de precipitação
            para determinada região. Os valores -99 delimitam a região válida para as previsões.
            Formato do nome: ppi-CZ-1-201501-1-0-0.txt
                                      y   mo d h m
            y: ano (2015)
            mo: mês (1)
            d: dia (1 - 31)
            h: hora (0 - 23)  
            m: minuto (00 - 50, a cada 10 minutos)
        """
        data = {}
        for day in range(0,31):
            for hour in range(0,24):
                for minutes in range(0,6):
                    data[(day*144)+(hour*6)+(minutes)] = np.genfromtxt("01/ppi-CZ-1-201501-"+str(day+1)+"-"+str(hour)+"-"+str(minutes*10)+".txt")

        return(data)

    def dataValidation(data):
        """
        Parameters:
        -----------
            Matrizes retornadas pela função reading.files()
        Returns:
        --------
            Array unidimensional contendo os valores válidos (excluindo os valores -99)
            e um array contendo a posição dos dados na matriz original.
        """
        new_data = {}
        new_idxs = {}
        aux_data = np.empty((0,0))
        idx = np.empty((0,2))
        for keys in range(len(data)):
            for rows in range(len(data[keys][:,0])):
                for columns in range(len(data[keys][0,:])):
                    if (data[keys][rows,columns] != -99):
                        aux_data = np.append(aux_data,data[keys][rows,columns])
                        idx = np.append(idx,[[rows,columns]],axis=0)
            new_data[keys] = aux_data
            new_idxs[keys] = idx
            aux_data = np.empty((0,0))
            idx = np.empty((0,2))
        
        return(new_data,new_idxs[0])

    def timeSeriesGeneration(valid_data):
        """
        Parameters:
        -----------
            Arrays unidimensionais contendo os valores estimados das previsões
        Returns:
        --------
            Série temporal associada a cada ponto válido (diferente de -99).
            A série temporal 0 será composta por todos os valores na posição 0 dos 4464 arrays, e assim
            em diante. O resultado são 587 séries temporais de tamanho 4464.
        """
        time_series = {}
        new_data = np.empty((0,0))
        for time in range(len(valid_data[0])):
            for data in range(len(valid_data)):
                new_data = np.append(new_data,valid_data[data][time])
            time_series[time] = new_data
            new_data = np.empty((0,0))
        
        return(time_series)

class network():
    
    def geo_graph(adj):
        """
        Parameters:
        -----------
        
        Returns:
        --------
        """
        geo_graph = igraph.Graph()
        geo_graph.add_vertices(587) # Número de séries temporais
        for i in range(geo_graph.vcount()):
            geo_graph.vs[i]["label"] = i # Enumerando os nós de 0 a 586
        geo_graph.add_edges(adj)

    def graphs(data):
        """
        Parameters:
        -----------
            Lista de valores inteiros que correspondem aos nós de uma rede. Os valores devem ser conectados
            em sequência.
            Exemplo:
                data = (1,2,8,4,5,2,7,9,4)
                adjacency_list from data: (1,2),(2,8),(8,4),(4,5),(5,2),(2,7),(7,9),(9,4)
            Assim, um grafo seria construído usando o conjunto data como os nós da rede e o conjunto
            adjacency_list seria responsável pela conexão dos nós.
        
        Returns:
        --------
            graph_metrics - Dicionário normalizado contendo as medidas average degree, average betweenness,
            average closeness, density e diameter.
            graph_degree - Array contendo o degree de cada nó da rede.
            graph_betweenness - Array contendo o betweenness de cada nó da rede.
            graph_closeness - Array contendo o closeness de cada nó da rede.
        """

        # Graph creation from decimal integer array

        graph = igraph.Graph()

        #graph.add_vertices(max(data)+1)
        graph.add_vertices(len(data))

        #adjacency_list = np.empty((0,0), int)
        aux_list = []
        adjacency_list = []

        for i in range(len(data)):
            if not(len(data)-1 == i):
                aux_list.append([data[i],data[i+1]])

        for i,j in aux_list:
            if not (([i,j] in adjacency_list) or ([j,i] in adjacency_list) or (i == j)):
                adjacency_list.append([i,j])

        #for i in range(max(data)+1):
        for i in range(len(data)):
            graph.vs[i]["label"] = i

        graph.add_edges(adjacency_list) # Full graph, with disconnected nodes

        # Only nodes with degree > 0

        disconnected = np.empty((0,0), int)
        for i in range(graph.vcount()):
            if (graph.vs[i].degree() == 0):
                disconnected = np.append(disconnected, i)

        graph.delete_vertices(disconnected)
        
        graph_metrics,graph_degree,graph_betweenness,graph_closeness = network.metrics(graph)
        
        return(graph_metrics,graph_degree,graph_betweenness,graph_closeness)
        
    def metrics(g):
        """
        Parameters:
        -----------
            Grafo criado na biblioteca igraph
            
        Returns:
        --------
            graph_metrics - Dicionário normalizado contendo as medidas average degree, average betweenness,
            average closeness, density e diameter.
            graph_degree - Array contendo o degree de cada nó da rede.
            graph_betweenness - Array contendo o betweenness de cada nó da rede.
            graph_closeness - Array contendo o closeness de cada nó da rede.            
        """

        # Degree and Average Degree
        degree_sequence = [round(i,4) for i in g.degree()]
        avg_degree = round((sum(degree_sequence)/len(degree_sequence)),4)

        # Diameter
        diameter = round(g.diameter(),4)

        # Betweenness and Average Betweenness
        betweenness_sequence = [round(i,4) for i in g.betweenness()]
        avg_betweenness = round((sum(betweenness_sequence)/len(betweenness_sequence)),4)

        # Closeness and Average Closeness
        closeness_sequence = [round(i,4) for i in g.closeness()]
        avg_closeness = round(sum(closeness_sequence)/len(closeness_sequence),4)

        # Density
        density = round(g.density(),4)

        metrics_dict = {
            'Avg_Betweenness': avg_betweenness,
            'Avg_Closeness': avg_closeness,
            'Avg_Degree': avg_degree,
            'Density': density,
            'Diameter': diameter
        }

        return(metrics_dict,degree_sequence,betweenness_sequence,closeness_sequence)

class algorithms():
    
    def DCSD(time_series,n): # Dynamical Characterization with Symbolic Dynamics - DCSD
        '''
        Parameters:
        -----------
            time_series - Série temporal a ser convertida em grafo.
            n - tamanho da palavra binária usada na conversão binário/decimal
            
            Exemplo de aplicação do algoritmo:
                1. Definir o valor máximo e mínimo de uma série temporal:
                    1.1. x = [5,8,9,12,10,7,4,3,5,7,11]
                    1.2. Max: 12
                         Min: 3
                2. Definir o centro da série temporal:
                    2.1. centro: (12+3)/2 = 7.5
                3. Gerar um array binário (b), tal como se segue:
                    3.1. x(i) >= centro, então b(i) = 1
                    3.2. x(i) < centro, então b(i) = 0
                    3.3. b = [0,1,1,1,1,0,0,0,0,0,1]
                4. Conversão da série binária em uma série decimal:
                    4.1. A conversão será feita através de palavras binárias de tamanho N.
                    4.2. O grupamento de tamanho 4 terá um comportamento deslizante, sempre com um passo
                    à direita:
                        4.2.1. Assumindo N = 4:
                                [|0,1,1,1|,1,0,0,0,0,0,1]
                                     7
                                [0,|1,1,1,1|,0,0,0,0,0,1]
                                       15
                                [0,1,|1,1,1,0|,0,0,0,0,1]
                                         14
                                [0,1,1,|1,1,0,0|,0,0,0,1]
                                          12
                                [0,1,1,1,|1,0,0,0|,0,0,1]
                                             8
                                [0,1,1,1,1,|0,0,0,0|,0,1]
                                               0
                                [0,1,1,1,1,0,|0,0,0,0|,1]
                                                 0
                                [0,1,1,1,1,0,0,|0,0,0,1|]
                                                   1

                                Nova série decimal inteira = [7,15,14,12,8,0,0,1]

                5. Cada valor da nova série gerada corresponde a um nó e a conexão entre esses nós se dá
                através da sequência dos valores, como se segue: (7,15), (15,14), (14,12), e assim por diante.
            
            Returns:
            --------
                graph_metrics - Dicionário normalizado contendo as medidas average degree, average betweenness,
                average closeness, density e diameter.
                graph_degree - Array contendo o degree de cada nó da rede.
                graph_betweenness - Array contendo o betweenness de cada nó da rede.
                graph_closeness - Array contendo o closeness de cada nó da rede.

            References:
            -----------
                FREITAS, Vander LS; LACERDA, Juliana C.; MACAU, Elbert EN.
                Complex Networks Approach for Dynamical Characterization of Nonlinear Systems.
                International Journal of Bifurcation and Chaos, v. 29, n. 13, p. 1950188, 2019.
            '''
        center = (max(time_series)+min(time_series))/2

        # Converting time_series to binary

        binary_array = np.empty((0,0), int)

        for i in time_series:
            if i >= center:
                binary_array = np.append(binary_array,1)
            else:
                binary_array = np.append(binary_array,0)

        # Converting the binary array to decimal (integer)

        decimal_array = np.empty((0,0), int)

        for i in range(len(binary_array)-n+1): 
            word = binary_array[i:i+n]
            string_word = ''
            for j in range(len(word)): 
                string_word = string_word + str(word[j])
            # Converting to decimal
            decimal_number = int(string_word,2)
            decimal_array = np.append(decimal_array,decimal_number)     

        graph_metrics_DCSD,graph_degree_DCSD,graph_betweenness_DCSD,graph_closeness_DCSD = network.graphs(decimal_array)

        return(graph_metrics_DCSD,graph_degree_DCSD,graph_betweenness_DCSD,graph_closeness_DCSD)

    def DCTIF(time_series, n): # Dynamical Characterization with Top Integral Function - DCTIF
        """
        Parameters:
        -----------
            time_series - Série temporal a ser convertida em grafo.
            n - Tamanho do intervalo em que os dados serão mapeados.            
            Exemplo:
                1. A função DCTIF é definida como: Yk = [N · xk ] = min{i ∈ Z | N · xk ≤ i}
                    1.1. Exemplo: x = [0.467, 1.0, 0.933, 0.8, 0.533, 0.0, 0.0, 0.067, 0.0]
                    1.2. Usando N = 5
                    1.3. Yk[0] = min{i ∈ Z | 5 · 0.467 ≤ i} = min{i ∈ Z | 2.335 ≤ i} = 3
                    1.4. Yk = [3,5,5,4,3,1,1,1,1]
                    1.5. Cada valor da nova série gerada corresponde a um nó e a conexão entre eles se dá
                    através da sequência dos valores, como se segue: (3,5), (5,5), (5,4), e assim em diante.
            
        Returns:
        --------
            graph_metrics - Dicionário normalizado contendo as medidas average degree, average betweenness,
            average closeness, density e diameter.
            graph_degree - Array contendo o degree de cada nó da rede.
            graph_betweenness - Array contendo o betweenness de cada nó da rede.
            graph_closeness - Array contendo o closeness de cada nó da rede. 
            
        References:
        -----------
            FREITAS, Vander LS; LACERDA, Juliana C.; MACAU, Elbert EN.
            Complex Networks Approach for Dynamical Characterization of Nonlinear Systems.
            International Journal of Bifurcation and Chaos, v. 29, n. 13, p. 1950188, 2019.
        """
        # Normalizing the data
        time_series_normalized = np.empty((0,0), float)
        for i in time_series:
            time_series_normalized = np.append(time_series_normalized, ((i-min(time_series))/(max(time_series)-min(time_series))))    

        nodes_array = np.empty((0,0),int)
        for i in time_series_normalized:
            if (i*n == 0):
                nodes_array = np.append(nodes_array,int(1))
            elif (round(i*n) - i*n >= 0.0):
                nodes_array = np.append(nodes_array,int(round(i*n)))
            else:
                nodes_array = np.append(nodes_array,int(round(i*n) + 1.0))

        graph_metrics_DCTIF,graph_degree_DCTIF,graph_betweenness_DCTIF,graph_closeness_DCTIF = network.graphs(nodes_array)

        return(graph_metrics_DCTIF,graph_degree_DCTIF,graph_betweenness_DCTIF,graph_closeness_DCTIF)

    def VG(time_series):
        """
        Parameters:
        -----------
            Uma série temporal que será transformada em um grafo do tipo natural visibility graph. Cada ponto
            da série será convertido em um nó, sendo que a conexão entre os nós é definida de acordo com um 
            critério de visibilidade.
            Exemplo:
                Critério de visibilidade: yc < yb + (ya - yb)*((tb - tc)/(tb - ta))
                    Dois pontos quaisquer em uma série temporal, (ya,ta) e (yb,tb), serão conectados no grafo,
                    se, e somente se, um terceiro ponto, (yc,tc), colocado entre eles for capaz de satisfazer
                    o critério de visibilidade.
                y = [5,8,9,12,10,7,4,3,5,7,11]
                t = [0,1,2,3,4,5,6,7,8,9,10] -> eixo temporal 
                Avaliando os pontos ya = 5 e yb = 8:
                - Não há pontos intermediários, yc = 0
                - Critério:
                    0 < 8 + (5-8)*((1-0)/(1-0))
                    0 < 8 + (-3)*1 
                    0 < 5
                    Logo, os pontos 5 e 8 serão conectados.
                Avaliando os pontos ya = 9 e yb = 10:
                - Ponto intermediário: yc = 12
                - Critério:
                    12 < 10 + (9 - 10)*((4-3)/(4-2))
                    12 < 10 + (-1)*(1/2)
                    12 < 10 - 1/2
                    12 < 9.5
                    Condição não satisfeita. Sendo assim, os pontos 9 e 10 não seriam conectados na rede.
                
        Returns:
        --------
            graph_metrics - Dicionário normalizado contendo as medidas average degree, average betweenness,
            average closeness, density e diameter.
            graph_degree - Array contendo o degree de cada nó da rede.
            graph_betweenness - Array contendo o betweenness de cada nó da rede.
            graph_closeness - Array contendo o closeness de cada nó da rede.

        References:
        -----------
            LACASA, Lucas et al. 
            From time series to complex networks: The visibility graph.
            Proceedings of the National Academy of Sciences, v. 105, n. 13, p. 4972-4975, 2008.
        """
        vg_igraph = ts2vg.NaturalVisibilityGraph(time_series).as_igraph()
        graph_metrics_VG,graph_degree_VG,graph_betweenness_VG,graph_closeness_VG = network.metrics(vg_igraph)
        
        return(graph_metrics_VG,graph_degree_VG,graph_betweenness_VG,graph_closeness_VG)

class saving():
    
    def saving_metrics(vg_metrics,dctif_metrics,dcsd_metrics):
        """
        Parameters:
        -----------
            Dicionários contendo as métricas nonrmalizadas average degree, average betweenness, 
            average closeness,density e diameter para cada um dos três algoritmos de conversão de 
            séries temporais em grafos.
                
            Os dados são salvos no formato csv.
            
        """
        VG_metrics_df = pd.DataFrame(data=vg_metrics.values())
        DCSD_metrics_df = pd.DataFrame(data=dcsd_metrics.values())
        DCTIF_metrics_df = pd.DataFrame(data=dctif_metrics.values())

        VG_metrics_df.to_csv('VG-metrics.csv',index=False)
        DCSD_metrics_df.to_csv('DCSD-metrics.csv',index=False)
        DCTIF_metrics_df.to_csv('DCTIF-metrics.csv',index=False)

        
    def saving_degree(vg_degree,dctif_degree,dcsd_degree):
        """
        Parameters:
        -----------
            Sequência de graus de cada grafo para os 3 algoritmos testados.
               
            Os dados são salvos no formato csv.
        """
        for i in range(587):
            if (len(dcsd_degree[i]) != 4464):
                for j in range((4464-len(dcsd_degree[i]))):
                    dcsd_degree[i].append(0)
            if (len(dctif_degree[i]) != 4464):
                for k in range((4464-len(dctif_degree[i]))):
                    dctif_degree[i].append(0)

        VG_degrees_df = pd.DataFrame(data=vg_degree)
        DCSD_degrees_df = pd.DataFrame(data=dcsd_degree)
        DCTIF_degrees_df = pd.DataFrame(data=dctif_degree)

        VG_degrees_df.to_csv('VG-degrees.csv',index=False)
        DCSD_degrees_df.to_csv('DCSD-degrees.csv',index=False)
        DCTIF_degrees_df.to_csv('DCTIF-degrees.csv',index=False)

    def saving_betweenness(vg_betweenness,dctif_betweenness,dcsd_betweenness):
        """
        Parameters:
        -----------
            Sequência dos valores de betweenness de cada grafo para os 3 algoritmos testados.
                
            Os dados são salvos no formato csv.
        """
        for i in range(587):
            if (len(dcsd_betweenness[i]) != 4464):
                for j in range((4464-len(dcsd_betweenness[i]))):
                    dcsd_betweenness[i].append(0)
            if (len(dctif_betweenness[i]) != 4464):
                for k in range((4464-len(dctif_betweenness[i]))):
                    dctif_betweenness[i].append(0)

        vg_betweenness_df = pd.DataFrame(data=vg_betweenness)
        dcsd_betweenness_df = pd.DataFrame(data=dcsd_betweenness)
        dctif_betweenness_df = pd.DataFrame(data=dctif_betweenness)

        vg_betweenness_df.to_csv('VG-betweenness.csv',index=False)
        dcsd_betweenness_df.to_csv('DCSD-betweenness.csv',index=False)
        dctif_betweenness_df.to_csv('DCTIF-betweenness.csv',index=False)
        
    def saving_closeness(vg_closeness,dctif_closeness,dcsd_closeness):
        """
        Parameters:
        -----------
            Sequência dos valores de closeness de cada grafo para os 3 algoritmos testados.
                
            Os dados são salvos no formato csv.
        """
        for i in range(587):
            if (len(dcsd_closeness[i]) != 4464):
                for j in range((4464-len(dcsd_closeness[i]))):
                    dcsd_closeness[i].append(0)
            if (len(dctif_closeness[i]) != 4464):
                for k in range((4464-len(dctif_closeness[i]))):
                    dctif_closeness[i].append(0)

        vg_closeness_df = pd.DataFrame(data=vg_closeness)
        dcsd_closeness_df = pd.DataFrame(data=dcsd_closeness)
        dctif_closeness_df = pd.DataFrame(data=dctif_closeness)

        vg_closeness_df.to_csv('VG-closeness.csv',index=False)
        dcsd_closeness_df.to_csv('DCSD-closeness.csv',index=False)
        dctif_closeness_df.to_csv('DCTIF-closeness.csv',index=False)

        
class comparing():
    
    def __init__(self, graph_metrics):
        """
        Parameters:
        -----------
            Array de métricas que será usado como parâmetro para o restante das funções dessa classe.
            São recebidos tanto os feature arrays, com métricas combinadas, quanto os sequence arrays, como é
            o caso da sequência de graus.
            As métricas enviadas são as pertencentes a todos os grafos. Sendo assim, self.metrics_array
            receberá um array de tamanho 587, onde cada posição terá um array de métricas correspondente a
            um grafo.
            As funções de distância/similaridade irão computar a medida entre todos os pares (a,b) de métricas,
            com a e b variando de 0 até 586, apenas desconsiderando casos como: 
                - a = b
                - (b,a) se já existir um valor calculado para o par (a,b)
        """
        self.metrics_array = graph_metrics
    
    def jensenshannon(self):
        """
        Returns:
        --------
            Retorna a o valor da distância de Jensen-Shannon, no formato de um dicionário, entre todos os 
            pares de grafos.
            A função computa a raiz quadrada da divergência de Jensen-Shannon:
                JSD(P||Q) = ((D(P||M) + D(Q||M))/2)^(1/2)
                Onde:
                    D é a divergência de Kullback-Leibler 
                    M = (P + Q)/2 
        
        Reference:
        ----------
            SciPy community. SciPy Reference Guide. <https://docs.scipy.org/doc/scipy/scipy-ref-1.5.4.pdf>, 
            v. 1.5.4, p. 2205.
                
        """
        jensenshannon_dist = {}
        ctrl = 1
        for i in range(len(self.metrics_array)):
            for j in range(ctrl,len(self.metrics_array)):
                jensenshannon_dist[i,j] = distance.jensenshannon(self.metrics_array[i],self.metrics_array[j])
            ctrl += 1
        return(jensenshannon_dist)    
    
    
    def euclidean(self):
        """
        Returns:
        --------
            Retorna a o valor da distância Euclidiana, no formato de um dicionário, entre todos os 
            pares de grafos.
            Considerando os vetores, a e b, a distância euclidiana é calculada como:
                e_dist(a,b) = ||a - b||
        
        Reference:
        ----------
            SciPy community. SciPy Reference Guide. <https://docs.scipy.org/doc/scipy/scipy-ref-1.5.4.pdf>, 
            v. 1.5.4, p. 2204-2205.
            
        """
        euclidean_dist = {}
        ctrl = 1
        for i in range(len(self.metrics_array)):
            for j in range(ctrl,len(self.metrics_array)):
                euclidean_dist[i,j] = distance.euclidean(self.metrics_array[i],self.metrics_array[j])
            ctrl += 1
        return(euclidean_dist)

    def manhattan(self):
        """    
        Returns:
        --------
            Retorna a o valor da distância de Manhattan (Cityblock), no formato de um dicionário, entre 
            todos os pares de grafos.
            Considerando os vetores a e b, a distância de manhattan é calculada como:
                m_dist(a,b) = Σ(|ai - bi|)
                i deve ser iterado a fim de percorrer todos os elementos de cada vetor.

        Reference:
        ----------
            SciPy community. SciPy Reference Guide. <https://docs.scipy.org/doc/scipy/scipy-ref-1.5.4.pdf>, 
            v. 1.5.4, p. 2203.
        """
        manhattan_dist = {}
        ctrl = 1
        for i in range(len(self.metrics_array)):
            for j in range(ctrl,len(self.metrics_array)):
                manhattan_dist[i,j] = distance.cityblock(self.metrics_array[i],self.metrics_array[j])
            ctrl += 1
        return(manhattan_dist)

    def canberra(self):
        """   
        Returns:
        --------
            Retorna a o valor da distância de Canberra, no formato de um dicionário, entre 
            todos os pares de grafos.
            Considerando os vetores a e b, a distância de canberra é calculada como:
                c_dist(a,b) = Σ((|ai - bi|)/(|ai|+|bi|))
                i deve ser iterado a fim de percorrer todos os elementos de cada vetor

        Reference:
        ----------
            SciPy community. SciPy Reference Guide. <https://docs.scipy.org/doc/scipy/scipy-ref-1.5.4.pdf>, 
            v. 1.5.4, p. 2202.
        """
        canberra_dist = {}
        ctrl = 1
        for i in range(len(self.metrics_array)):
            for j in range(ctrl,len(self.metrics_array)):
                canberra_dist[i,j] = distance.canberra(self.metrics_array[i],self.metrics_array[j])
            ctrl += 1
        return(canberra_dist)

    def cosine(self):
        """    
        Returns:
        --------
            Retorna a o valor da distância baseada em cosseno, no formato de um dicionário, entre 
            todos os pares de grafos.
            Considerando os vetores a e b, o cosseno entre esses dois vetores pode ser obtido pela
            seguinte relação:
                cos(θ) = (a·b)/(||a||*||b||)
                cos_dist = 1 - cos(θ)

                (a·b) é o produto escalar entre os dois vetores.
                
        Reference:
        ----------
            SciPy community. SciPy Reference Guide. <https://docs.scipy.org/doc/scipy/scipy-ref-1.5.4.pdf>, 
            v. 1.5.4, p. 2204.
        """
        cosine_sim = {}
        ctrl = 1
        for i in range(len(self.metrics_array)):
            for j in range(ctrl,len(self.metrics_array)):
                cosine_sim[i,j] = distance.cosine(self.metrics_array[i],self.metrics_array[j])
            ctrl += 1
        return(cosine_sim)


# -------------------------------------------------------------------------------------------------------------    

# Reading files and time series generation

In [48]:
# Estimated execution time: 64,39s
init = time.time()

# Time series
files = reading.files()
validated_files,position = reading.dataValidation(files) # removing -99 values
meteorological_time_series = reading.timeSeriesGeneration(validated_files)

end = time.time()
print(end-init)

58.618616342544556


# Creating graphs

In [ ]:
# Estimated execution time: 5357,62s
init = time.time()

vg_metrics = {}
vg_degree = {}
vg_betweenness = {}
vg_closeness = {}

dctif_metrics = {}
dctif_degree = {}
dctif_betweenness = {}
dctif_closeness = {}

dcsd_metrics = {}
dcsd_degree = {}
dcsd_betweenness = {}
dcsd_closeness = {}

for idx in range(len(meteorological_time_series)):
    print(idx,"de",(len(meteorological_time_series)-1))
    vg_metrics[idx],vg_degree[idx],vg_betweenness[idx],vg_closeness[idx] = algorithms.VG(meteorological_time_series[idx])
    dcsd_metrics[idx],dcsd_degree[idx],dcsd_betweenness[idx],dcsd_closeness[idx] = algorithms.DCSD(meteorological_time_series[idx],10)
    dctif_metrics[idx],dctif_degree[idx],dctif_betweenness[idx],dctif_closeness[idx] = algorithms.DCTIF(meteorological_time_series[idx],50)
    
end = time.time()
print(end-init)

# Saving metrics

In [50]:
# Estimated excecution time: 18,96s
init = time.time()

saving.saving_metrics(vg_metrics,dctif_metrics,dcsd_metrics)
saving.saving_degree(vg_degree,dctif_degree,dcsd_degree)
saving.saving_betweenness(vg_betweenness,dctif_betweenness,dcsd_betweenness)
saving.saving_closeness(vg_closeness,dctif_closeness,dcsd_closeness)

end = time.time()
print(end-init)

17.807220697402954


# Calculando as distâncias 

## Visibility Graphs - VG

In [16]:
# VG
# Tempo de execução: 115s
init = time.time()

vg_metrics_df = pd.read_csv("VG-metrics.csv") # Cada linha do DataFrame é um vetor de características
vg_degrees_df = pd.read_csv("VG-degrees.csv") # Cada coluna representa a distribuição de graus de uma rede
vg_betweenness_df = pd.read_csv("VG-betweenness.csv") # Cada coluna representa a distribuição de betweenness de uma rede
vg_closeness_df = pd.read_csv("VG-closeness.csv") # Cada coluna representa a distribuição de closeness de uma rede

vg_metrics_array = vg_metrics_df.to_numpy() # Cada posição do array possui um vetor de características
vg_degrees_array = vg_degrees_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de graus
vg_betweenness_array = vg_betweenness_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de betweenness
vg_closeness_array = vg_closeness_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de closeness

# Calculando as métricas entre todos os vetores de características.

dist_vg_metrics = comparing(vg_metrics_array)


euclidean_dist_vg_metrics = dist_vg_metrics.euclidean() # Não é capaz de captar correlação
manhattan_dist_vg_metrics = dist_vg_metrics.manhattan() # Não é capaz de captar correlação
canberra_dist_vg_metrics = dist_vg_metrics.canberra() # Não é capaz de captar correlação
cosine_dist_vg_metrics = dist_vg_metrics.cosine() # Não é capaz de captar correlação
jensenshannon_dist_vg_metrics = dist_vg_metrics.jensenshannon() # Não é capaz de captar correlação 

# Calculando as métricas entre todos os vetores com distribuição de graus.

dist_vg_degrees = comparing(vg_degrees_array)

euclidean_dist_vg_degrees = dist_vg_degrees.euclidean() # Há correlação, mas a definição do limiar é problemática
manhattan_dist_vg_degrees = dist_vg_degrees.manhattan() # Limiar consistente. Teste em: <18000
canberra_dist_vg_degrees = dist_vg_degrees.canberra() # Limiar pouco consistente. Teste em: <330
cosine_dist_vg_degrees = dist_vg_degrees.cosine() # Limiar consistente. Teste em: <0.27
jensenshannon_dist_vg_degrees = dist_vg_degrees.jensenshannon() # Limiar consistente. Teste em: <0.27

# Calculando as métricas entre todos os vetores com a sequência de betweenness.

dist_vg_betweenness = comparing(vg_betweenness_array)

euclidean_dist_vg_betweenness = dist_vg_betweenness.euclidean() #
manhattan_dist_vg_betweenness = dist_vg_betweenness.manhattan() # 
canberra_dist_vg_betweenness = dist_vg_betweenness.canberra() # 
cosine_dist_vg_betweenness = dist_vg_betweenness.cosine() # 
jensenshannon_dist_vg_betweenness = dist_vg_betweenness.jensenshannon() #


# Calculando as métricas entre todos os vetores com a sequência de closeness.

dist_vg_closeness = comparing(vg_closeness_array)

euclidean_dist_vg_closeness = dist_vg_closeness.euclidean() # 
manhattan_dist_vg_closeness = dist_vg_closeness.manhattan() #
canberra_dist_vg_closeness = dist_vg_closeness.canberra() #
cosine_dist_vg_closeness = dist_vg_closeness.cosine() # 
jensenshannon_dist_vg_closeness = dist_vg_closeness.jensenshannon() # 


end = time.time()
print(end-init)

241.66877150535583


## Dynamic Characterization using Symbolic Dynamics - DCSD

In [38]:
# DCSD
# Tempo de execução: 177s
init = time.time()

dcsd_metrics_df = pd.read_csv("DCSD-metrics.csv") # Cada linha do DataFrame é um vetor de características
dcsd_degrees_df = pd.read_csv("DCSD-degrees.csv") # Cada coluna representa a distribuição de graus de uma rede
dcsd_betweenness_df = pd.read_csv("DCSD-betweenness.csv") # Cada coluna representa a distribuição de betweenness de uma rede
dcsd_closeness_df = pd.read_csv("DCSD-closeness.csv") # Cada coluna representa a distribuição de closeness de uma rede

dcsd_metrics_array = dcsd_metrics_df.to_numpy() # Cada posição do array possui um vetor de características
dcsd_degrees_array = dcsd_degrees_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de graus
dcsd_betweenness_array = dcsd_betweenness_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de betweenness
dcsd_closeness_array = dcsd_closeness_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de closeness

# Calculando as métricas entre todos os vetores de características.

dist_dcsd_metrics = comparing(dcsd_metrics_array)


euclidean_dist_dcsd_metrics = dist_dcsd_metrics.euclidean() # 
manhattan_dist_dcsd_metrics = dist_dcsd_metrics.manhattan() # 
canberra_dist_dcsd_metrics = dist_dcsd_metrics.canberra() # 
cosine_dist_dcsd_metrics = dist_dcsd_metrics.cosine() # 
jensenshannon_dist_dcsd_metrics = dist_dcsd_metrics.jensenshannon() # 

# Calculando as métricas entre todos os vetores com distribuição de graus.

dist_dcsd_degrees = comparing(dcsd_degrees_array)

euclidean_dist_dcsd_degrees = dist_dcsd_degrees.euclidean() # 
manhattan_dist_dcsd_degrees = dist_dcsd_degrees.manhattan() # 
canberra_dist_dcsd_degrees = dist_dcsd_degrees.canberra() # 
cosine_dist_dcsd_degrees = dist_dcsd_degrees.cosine() # 
jensenshannon_dist_dcsd_degrees = dist_dcsd_degrees.jensenshannon() # 

# Calculando as métricas entre todos os vetores com a sequência de betweenness.

dist_dcsd_betweenness = comparing(dcsd_betweenness_array)

euclidean_dist_dcsd_betweenness = dist_dcsd_betweenness.euclidean() #
manhattan_dist_dcsd_betweenness = dist_dcsd_betweenness.manhattan() # 
canberra_dist_dcsd_betweenness = dist_dcsd_betweenness.canberra() # 
cosine_dist_dcsd_betweenness = dist_dcsd_betweenness.cosine() # 
jensenshannon_dist_dcsd_betweenness = dist_dcsd_betweenness.jensenshannon() #


# Calculando as métricas entre todos os vetores com a sequência de closeness.

dist_dcsd_closeness = comparing(dcsd_closeness_array)

euclidean_dist_dcsd_closeness = dist_dcsd_closeness.euclidean() # 
manhattan_dist_dcsd_closeness = dist_dcsd_closeness.manhattan() #
canberra_dist_dcsd_closeness = dist_dcsd_closeness.canberra() #
cosine_dist_dcsd_closeness = dist_dcsd_closeness.cosine() # 
jensenshannon_dist_dcsd_closeness = dist_dcsd_closeness.jensenshannon() # 


end = time.time()
print(end-init)

176.64785528182983


## Dynamic Characterization using Top Integral Function - DCTIF

In [4]:
# DCTIF
# Tempo de execução: 159s
init = time.time()

dctif_metrics_df = pd.read_csv("DCTIF-metrics.csv") # Cada linha do DataFrame é um vetor de características
dctif_degrees_df = pd.read_csv("DCTIF-degrees.csv") # Cada coluna representa a distribuição de graus de uma rede
dctif_betweenness_df = pd.read_csv("DCTIF-betweenness.csv") # Cada coluna representa a distribuição de betweenness de uma rede
dctif_closeness_df = pd.read_csv("DCTIF-closeness.csv") # Cada coluna representa a distribuição de closeness de uma rede

dctif_metrics_array = dctif_metrics_df.to_numpy() # Cada posição do array possui um vetor de características
dctif_degrees_array = dctif_degrees_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de graus
dctif_betweenness_array = dctif_betweenness_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de betweenness
dctif_closeness_array = dctif_closeness_df.T.to_numpy() # Cada posição do array guarda o vetor com a distribuição de closeness

# Calculando as métricas entre todos os vetores de características.

dist_dctif_metrics = comparing(dctif_metrics_array)


euclidean_dist_dctif_metrics = dist_dctif_metrics.euclidean() # 
manhattan_dist_dctif_metrics = dist_dctif_metrics.manhattan() # 
canberra_dist_dctif_metrics = dist_dctif_metrics.canberra() # 
cosine_dist_dctif_metrics = dist_dctif_metrics.cosine() # 
jensenshannon_dist_dctif_metrics = dist_dctif_metrics.jensenshannon() # 

# Calculando as métricas entre todos os vetores com distribuição de graus.

dist_dctif_degrees = comparing(dctif_degrees_array)

euclidean_dist_dctif_degrees = dist_dctif_degrees.euclidean() # 
manhattan_dist_dctif_degrees = dist_dctif_degrees.manhattan() # 
canberra_dist_dctif_degrees = dist_dctif_degrees.canberra() # 
cosine_dist_dctif_degrees = dist_dctif_degrees.cosine() # 
jensenshannon_dist_dctif_degrees = dist_dctif_degrees.jensenshannon() # 

# Calculando as métricas entre todos os vetores com a sequência de betweenness.

dist_dctif_betweenness = comparing(dctif_betweenness_array)

euclidean_dist_dctif_betweenness = dist_dctif_betweenness.euclidean() #
manhattan_dist_dctif_betweenness = dist_dctif_betweenness.manhattan() # 
canberra_dist_dctif_betweenness = dist_dctif_betweenness.canberra() # 
cosine_dist_dctif_betweenness = dist_dctif_betweenness.cosine() # 
jensenshannon_dist_dctif_betweenness = dist_dctif_betweenness.jensenshannon() #


# Calculando as métricas entre todos os vetores com a sequência de closeness.

dist_dctif_closeness = comparing(dctif_closeness_array)

euclidean_dist_dctif_closeness = dist_dctif_closeness.euclidean() # 
manhattan_dist_dctif_closeness = dist_dctif_closeness.manhattan() #
canberra_dist_dctif_closeness = dist_dctif_closeness.canberra() #
cosine_dist_dctif_closeness = dist_dctif_closeness.cosine() # 
jensenshannon_dist_dctif_closeness = dist_dctif_closeness.jensenshannon() # 


end = time.time()
print(end-init)

152.17093443870544
